In [27]:
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import json # To load settings

# Packages for quantum stuff
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator, QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import FakeBrisbane
import qiskit
from qiskit import qasm3

In [28]:
def cost_func_estimator(params, ansatz, estimator):  # Removed hamiltonian argument
    # The Hamiltonian information should ideally be accessed via the ansatz
    # or passed if strictly needed, but Estimator needs the observable.
    # Rebuild observable based on ansatz qubits if needed, or use pre-built one
    # Assuming cost_hamiltonian is defined globally/accessible here for simplicity
    # apply_layout might be needed if the estimator doesn't handle it automatically
    # based on the transpiled ansatz. Let's assume estimator + transpiled ansatz handles layout.
    isa_observable = cost_hamiltonian.apply_layout(
        ansatz.layout
    )  # Apply layout based on *transpiled* ansatz

    pub = (ansatz, isa_observable, [params])  # Pass params in a list for V2 PUB
    job = estimator.run(pubs=[pub])  # Pass pubs as keyword arg

    # Handle potential errors during job execution
    try:
        results = job.result()[0]  # Get the first PubResult
        cost = results.data.evs[0]  # Get the first (and only) EV
    except Exception as e:
        print(f"Error getting result for params {params}: {e}")
        # Return a high cost or handle error appropriately for the optimizer
        return float("inf")

    # Ensure cost is a standard float for scipy.optimize
    cost_float = float(np.real(cost))
    objective_func_vals.append(cost_float)
    print(f"Params: {params}, Cost: {cost_float}")  # Add print for debugging
    return cost_float


def build_max_cut_paulis_unit_weights(graph_rx, num_qubits):
    pauli_list = []
    # We only need the nodes involved in the edge from edge_list()
    for u, v in graph_rx.edge_list():
        paulis = ["I"] * num_qubits
        paulis[u], paulis[v] = "Z", "Z"
        # --- Assign weight directly for MaxCut ---
        weight = 1.0
        # -----------------------------------------
        pauli_list.append(("".join(paulis)[::-1], weight))
    return pauli_list

In [29]:
INPUT_QASM_FILE = 'parameterized_circuit.qasm3'
INPUT_SETTINGS_FILE = 'qaoa_settings.json'

with open(INPUT_QASM_FILE, 'r') as f:
        qasm3_string = f.read()
# Use qiskit.qasm3 loader
loaded_circuit = qiskit.qasm3.loads(qasm3_string)
print("Circuit loaded successfully.")

with open(INPUT_SETTINGS_FILE, 'r') as f:
        settings = json.load(f)
print("Settings loaded successfully.")
# Extract relevant settings
opt_method = settings["optimizer_method"]
opt_maxiter = settings["optimizer_maxiter"]
opt_tol = settings["optimizer_tol"]
num_qubits_check = settings["num_qubits"] # For verification
initial_betas = settings["initial_betas"]
initial_gammas = settings["initial_gammas"]

print("Recreating graph and Hamiltonian...")
graph = rx.PyGraph()
graph.add_nodes_from(settings["graph_nodes"])
graph.add_edges_from([(u, v, 1.0) for u, v in settings["graph_edges"]])

init_params = [
    param for pair in zip(initial_betas, initial_gammas) for param in pair
]
print("Initial parameters set.")

max_cut_paulis = build_max_cut_paulis_unit_weights(graph, settings["num_qubits"])
cost_hamiltonian = SparsePauliOp.from_list(max_cut_paulis)
print("Cost Hamiltonian recreated.")

Circuit loaded successfully.
Settings loaded successfully.
Recreating graph and Hamiltonian...
Initial parameters set.
Cost Hamiltonian recreated.


In [30]:
# 4
fakeBackend = FakeBrisbane()
simulator = AerSimulator.from_backend(fakeBackend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=fakeBackend)
candidate_circuit = pm.run(loaded_circuit)
#candidate_circuit.draw()

KeyError: <Qubit uid=2010>

In [26]:
# 6 finding the optimal parameters for the QAOA circuit using a 'estimator'
objective_func_vals = []  # Global variable

estimator = Estimator(mode=simulator)
estimator.options.default_shots = 1000

print("Starting optimization...")
result = minimize(
    cost_func_estimator, #need to inlcude the paramters from circuit generation not just the circuit so that it can know what the cost estimator is
    init_params,
    args=(candidate_circuit, estimator),  # Pass transpiled circuit and estimator
    method="COBYLA",
    tol=1e-2,  # Tolerance for termination
    options={"maxiter": 100},  # Set max iterations to prevent running forever
)
print("Optimization Result:")
print(result)

# You can plot the objective function values if desired
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

Starting optimization...


ValueError: Length of ('ONEQ_MIXER_seq0_layer0', 'ONEQ_MIXER_seq0_layer1', 'ONEQ_MIXER_seq0_layer2', 'ONEQ_MIXER_seq1_layer0', 'ONEQ_MIXER_seq1_layer1', 'ONEQ_MIXER_seq1_layer2', 'ONEQ_MIXER_seq2_layer0', 'ONEQ_MIXER_seq2_layer1', 'ONEQ_MIXER_seq2_layer2', 'ONEQ_MIXER_seq3_layer0', 'ONEQ_MIXER_seq3_layer1', 'ONEQ_MIXER_seq3_layer2', 'ONEQ_MIXER_seq4_layer0', 'ONEQ_MIXER_seq4_layer1', 'ONEQ_MIXER_seq4_layer2', 'ONEQ_MIXER_seq5_layer0', 'ONEQ_MIXER_seq5_layer1', 'ONEQ_MIXER_seq5_layer2', 'TWOQ_COST_seq0_layer0', 'TWOQ_COST_seq0_layer1', 'TWOQ_COST_seq0_layer2', 'TWOQ_COST_seq1_layer0', 'TWOQ_COST_seq1_layer1', 'TWOQ_COST_seq1_layer2', 'TWOQ_COST_seq2_layer0', 'TWOQ_COST_seq2_layer1', 'TWOQ_COST_seq2_layer2', 'TWOQ_COST_seq3_layer0', 'TWOQ_COST_seq3_layer1', 'TWOQ_COST_seq3_layer2', 'TWOQ_COST_seq4_layer0', 'TWOQ_COST_seq4_layer1', 'TWOQ_COST_seq4_layer2', 'TWOQ_COST_seq5_layer0', 'TWOQ_COST_seq5_layer1', 'TWOQ_COST_seq5_layer2', 'TWOQ_COST_seq6_layer0', 'TWOQ_COST_seq6_layer1', 'TWOQ_COS